In [5]:
import contacts as cts
import imessages as im

In [6]:
contact = cts.filter_contacts(firstname='ishani')[0]
contact

{'id': 8,
 'firstname': 'ishani 🐿️',
 'lastname': None,
 'phone': '5107503277',
 'email': None}

In [7]:
im_contact = im.get_contacts(f"id LIKE '%{contact["phone"]}%' OR id LIKE '%{contact["email"]}%'")[0]
im_contact

{'ROWID': 60,
 'id': '+15107503277',
 'country': 'US',
 'service': 'iMessage',
 'uncanonicalized_id': '5107503277',
 'person_centric_id': None}

In [8]:
if '@' in im_contact['id']:
    contact['email'] = im_contact['id']
else:
    contact['phone'] = im_contact['id']

contact

{'id': 8,
 'firstname': 'ishani 🐿️',
 'lastname': None,
 'phone': '+15107503277',
 'email': None}

---

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from tools import *
import json

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def create_response(input_messages):
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=input_messages,
        tools=tools,
        temperature=0.7,
        max_output_tokens=2048,
        include=["web_search_call.action.sources"]
    )
    input_messages += response.output
    for output_item in response.output:
        if hasattr(output_item, "action"):
            for source in output_item.action.sources:
                print(f"Source: {source.url}")
        if output_item.type == "function_call":
            print(f"Calling {output_item.name}({json.loads(output_item.arguments)})")
            function_output = call_function(output_item.name, json.loads(output_item.arguments))
            input_messages.append({
                "type": "function_call_output",
                "call_id": output_item.call_id,
                "output": str(function_output)
            })
            return create_response(input_messages)
    return response, input_messages

In [27]:
system_prompt = "You are a helpful assistant that can answer questions and help with tasks."
messages = [{"role": "system", "content": [{"type": "input_text", "text": system_prompt}]}]

In [ ]:
user_input = "What is the current date and time?"
messages.append({"role": "user", "content": [{"type": "input_text", "text": user_input}]})

response, messages = create_response(messages)

if len(messages) > 20:
    messages = messages[-20:]

print(f"{response.output_text} ({response.usage.total_tokens} tokens)")

[{'content': [{'text': 'What is the current date and time?',
               'type': 'input_text'}],
  'role': 'user'}]
Calling get_current_date({})
[{'content': [{'text': 'What is the current date and time?',
               'type': 'input_text'}],
  'role': 'user'},
 ResponseFunctionToolCall(arguments='{}', call_id='call_lzh8juQ7abtG1mG6d8938Fz1', name='get_current_date', type='function_call', id='fc_68d1d70d6a708198a3e93d1d1704d6a609f1b934d89329b5', status='completed'),
 {'call_id': 'call_lzh8juQ7abtG1mG6d8938Fz1',
  'output': 'Monday September 22, 2025',
  'type': 'function_call_output'}]
Calling get_current_time({})
[{'content': [{'text': 'What is the current date and time?',
               'type': 'input_text'}],
  'role': 'user'},
 ResponseFunctionToolCall(arguments='{}', call_id='call_lzh8juQ7abtG1mG6d8938Fz1', name='get_current_date', type='function_call', id='fc_68d1d70d6a708198a3e93d1d1704d6a609f1b934d89329b5', status='completed'),
 {'call_id': 'call_lzh8juQ7abtG1mG6d8938Fz1',